In [1]:
print("loading modules")
import os
from time import time
import csv

import numpy as np 
import pandas as pd
pd.set_option("display.max_columns", 200)

# import xarray as xr
# import scipy.stats as stats
# import scipy.spatial as spatial
# from sklearn import preprocessing

# import mne
# from mne import channels
# from mne import time_frequency

# import h5py

# import glob
# # from psifr import fr

# import ptsa 
# #from ptsa.data.TimeSeriesX import TimeSeries
# from ptsa.data.timeseries import TimeSeries
# from ptsa.data import timeseries
# from ptsa.data.readers import BaseEventReader

# from ptsa.data.filters import MorletWaveletFilter
# from ptsa.data.filters import ButterworthFilter
import cmlreaders as cml
from repeat_sme import *

loading modules
loading modules


/home1/radrogue/.conda/envs/environmentname/lib/python3.7/site-packages/ptsa/data/readers/__init__.py:19: FutureWarning: PTSA readers will be removed in a future release. Please consider using the cmlreaders package instead: https://github.com/pennmem/cmlreaders
  FutureWarning


In [2]:
def get_powers(subject):
    bad = []
    data = cml.get_data_index(kind = 'r1'); data = data[data['experiment'] == 'RepFR1']
    sessions = data[data.subject == subject].session.unique()
    freqs = np.logspace(np.log10(3),np.log10(180), 24)
    for sess in sessions:
        try:
            print(subject, sess, 'loading...')
            r = cml.CMLReader(subject=subject, experiment='RepFR1', session=sess)
            evs = r.load('task_events')

            word = evs.query("type == 'WORD'")
            # word = word[word.list != 0]
            word = word[word.list != -999]
            word.list.unique()
            print(subject, sess, 'filtering...')
            f, o, n = filter_repetitions(word, word.list.unique()[1:])
            # get_raw_eeg(subject, session)
            pairs = r.load("pairs")

            filtered = pairs[(pairs["mni.region"].str.contains("temporal")) & pairs['mni.region'].
                            str.contains('Left')]

            eeg = r.load_eeg(scheme=filtered)
            channels = eeg.channels

            rel_start = 0/1000
            rel_stop = 1600/1000
            wave_number = 5
            half_wav = ((1000 / freqs.min()) * wave_number) * 0.5
            sr = float(eeg.samplerate)

            buffer_ms = (wave_number * (1000/freqs[0])) / 2
            buffer_samp = int(np.ceil(buffer_ms * (sr/1000.)))
            buffer_minimized = buffer_ms/1000
            path = '/scratch/radrogue/RepFR/{}/sme_pow_debug/session_{}/'.format(subject,sess)
            ep, enc_x, mne_evs = Create_Epoch(eeg.to_mne(), f, rel_start + (-1 * buffer_minimized), rel_stop + buffer_minimized)

            enc_eeg = ButterworthFilter(freq_range =[58., 62.], filt_type = 'stop', order = 4).filter(enc_x)
            print(subject, sess, 'calculating powers...')
            Channel_Powers(subject = subject, session = sess, channels_list = channels, eeg=enc_eeg, buffer_samp=buffer_samp, wave_number = wave_number, freqs=freqs, path=path, error_log_path = '', log = True)
        except Exception as e:
            bad.append(subject+str(sess))
    return bad

In [3]:
data = cml.get_data_index(kind = 'r1'); data = data[data['experiment'] == 'RepFR1']

contacts = []

for subject, df in data.groupby('subject'):
    for session in pd.unique(df['session']):
        r = cml.CMLReader(subject=subject, experiment='RepFR1', session=session)
        temp = r.load('contacts')
        temp['subject'] = pd.Series(subject, index=temp.index)
        temp['session'] = pd.Series(session, index=temp.index)
        contacts.append(temp)
contacts = pd.concat(contacts)
regions = ['temporal', 'frontal', 'parietal', 'amygdala', 'Hippocampus']
hemispheres = ['Right', 'Left', '']

results = pd.DataFrame(columns=['region', 'subjects'])
for h in hemispheres:
    for r in regions:    
        n = contacts[(contacts['mni.region'].str.contains(r)) & contacts['mni.region'].
                     str.contains(h)].subject.unique()
        results = results.append(pd.DataFrame(dict(region = h +' ' + r, subjects = [n]), index = [len(results)]))
# LTL = results[(results.hemisphere == 'Left') & (results.region == 'temporal')]

index = get_data_index('r1')
LTL_subs = results[results.region == 'Left temporal'].subjects.iloc[0]


In [4]:
import CMLDask
from dask.distributed import wait, as_completed, progress
client.shutdown()

/home1/radrogue/.local/lib/python3.7/site-packages/dask_jobqueue/core.py:20: FutureWarning: tmpfile is deprecated and will be removed in a future release. Please use dask.utils.tmpfile instead.
  from distributed.utils import tmpfile


NameError: name 'client' is not defined

In [5]:
client = CMLDask.new_dask_client("iEEG_power", "10GB")

Unique port for radrogue is 51417
{'dashboard_address': ':51417'}
To view the dashboard, run: 
`ssh -fN radrogue@rhino2.psych.upenn.edu -L 8000:192.168.86.142:51417` in your local computer's terminal (NOT rhino) 
and then navigate to localhost:8000 in your browser


In [11]:
futures = client.map(get_powers, LTL_subs)

distributed.client - ERROR - Failed to reconnect to scheduler after 30.00 seconds, closing client
_GatheringFuture exception was never retrieved
future: <_GatheringFuture finished exception=CancelledError()>
concurrent.futures._base.CancelledError


In [12]:
progress(futures)

VBox()

In [13]:
client.gather(futures)

[[],
 [],
 [],
 [1],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [0, 1, 2, 3, 4, 5],
 [],
 [],
 [3],
 [],
 []]

In [14]:
client.shutdown()

distributed.client - ERROR - Failed to reconnect to scheduler after 30.00 seconds, closing client
_GatheringFuture exception was never retrieved
future: <_GatheringFuture finished exception=CancelledError()>
concurrent.futures._base.CancelledError


In [3]:
def filter_repetitions(events, list_index):

    '''
    two functions that split recall events that contain repeated word presentations into a new events dataframe containing only 
    the first and second presentation of repeated words. Then additionally outputs two masks to additionally filter the already filtered 
    recalls by either their first and second presentation.

    This is to ideally split first/second presentations by recalled and not-recalled to calculate two different SMEs

    Parameters
    ----------

    events: encoding events ideally from ltpRepFR and a similar repeated word presentation experiment 

    list_index: list of all list values (in ltpepFr 1 through 25)

    Return
    -------

    filtered_events: events that have one time presented words and the third presentation of repeated words removed 

    first: array mask to only include the first presentation of filtered repeated encoding events 

    second: array mask to only include the second presentation of filtered repeated encoding events 

    '''

    def pull_first_and_second_rep_presentations(events, list_index):


        '''
        take in events and list_index structure from ltpRepFR data, filter events to only include first and second presentation of 
        second and three times repeated words -- excluding practice list (list 0) and all single presentation words 


        Parameters
        ----------

        events: ltpRepFR word events 

        list_index: list of all list values (in ltpRepFr 1 through 25)

        Return
        -------
        event_mask: array of True and False values for all events -- True events are those that are 2 or 3 times repeated words and only their first and second presentations 
        '''


        session_array = []
        session_array.extend(np.repeat(False, 27))

        for l in list_index:
            current_list = events[events['list'] == l] #interate one list at a time 
            non_repeat = list(current_list[current_list['repeats'] == 1].item_name)

            #dict to track number of presentations of a word during one list 
            pres_dict = {}
            for x in np.unique(current_list.item_name):
                pres_dict['{0}'.format(x)] = 0

            for word in current_list.item_name:

                #if a once repeated word -- append False value and increment that word by 1 presentation 
                if word in non_repeat:
                    session_array.append(False)
                    pres_dict[str(word)] += 1 

                elif word not in non_repeat:
                    #presentation value 
                    presentation = pres_dict[str(word)]

                    #if word has reached or is reaching second presentation -- append True value and increment that word by 1 presentation 
                    if presentation <= 2:
                        session_array.append(True) 
                        pres_dict[str(word)] += 1 

                    elif presentation > 2:
                        session_array.append(False)
                        pres_dict[str(word)] += 1 

        event_mask = np.asarray(session_array).flatten()

        if event_mask.shape[0] != 567: 
            print("missing "+str(567 - int(np.asarray(session_array).shape[0]))+' encoding events. Not Complete.')

        return event_mask



    def split_first_and_second_pres_events(events, list_index):
        '''
        After removing single presentation words and the third presentation of repeated words, 
        specifically split the results encoding event array by first and second presentations and create respective masks 


        Parameters
        -----------
        events: filtered events from "pull_first_and_second_rep_presentation function"

        list_index: list of all list values (in ltpRepFr 1 through 25)

        Return
        -------

        first_p_mask: mask to filter event array for only the first presentations of repeated words 

        second_p_masl: mask to filter event array for only the second presentations of repeated words 
        '''

        first_p = []

        for l in list_index:
            current_list = events[events['list'] == l]

            pres_dict = {}
            for x in np.unique(current_list.item_name):
                pres_dict['{0}'.format(x)] = 0

            for word in current_list.item_name:
                presentation = pres_dict[str(word)]

                if presentation < 1:
                    first_p.append(True)
                    pres_dict[str(word)] += 1 

                elif presentation >= 1:
                    first_p.append(False)
                    pres_dict[str(word)] += 1 


        first_p_mask = np.asarray(first_p)
        second_p_mask = ~first_p_mask


        return first_p_mask, second_p_mask 

    event_mask = pull_first_and_second_rep_presentations(events, list_index)
    print(len(events))
    filtered_events = events[event_mask]
    first, second = split_first_and_second_pres_events(filtered_events, list_index)

    return filtered_events, first, second 

In [4]:
def save_numpy(array, path, name):

        combine = (str(path)+str(name))

        np.save( combine , array)

        return 

In [5]:
def get_raw_eeg(subject, sess):
#### Change this function


        ''' pull raw eeg from ephys directories on rhino -- drop bad channels from bad channel.txt files and compute an average reference + highpass filter  '''


        eegpath = '/protocols/r1/subjects/%s/experiments/RepFR1/sessions/%d/ephys/current_processed/' % (subject, sess)

        eegfile = glob.glob(eegpath+'*.edf')
        if len(eegfile) != 1:
            raise Exception("Multiple eeg files")

        eegfile = eegfile[0]
        print("does this try a high-pass??? ")
        raw = mne.io.read_raw_edf(eegfile, eog=['EXG1', 'EXG2', 'EXG3', 'EXG4'],
                                      misc=['EXG5', 'EXG6', 'EXG7', 'EXG8'],
                                      stim_channel='Status',               
                                      preload=True) # needs to be true for 0.1Hz high-pass filter to work  

#         badchanfile = glob.glob(eegpath+'*_bad_chan*.txt')
#         if len(badchanfile) > 0:
#             with open(badchanfile[0], 'r') as f:
#                 bad = [s.strip() for s in f.readlines()]
#             raw.info['bad'] = bad 


        return raw 

In [6]:
def Create_Epoch(eeg, events, start, stop):
    '''
    convert raw eeg to an MNE epoch and then convert to a TimeSeries object for better data manipulation
    runs average reference and resamples data
    '''
    mne_events = np.zeros((len(events), 3), dtype = int)
    mne_events[:,0] = [o for i, o in enumerate(events['eegoffset'])]
#     print("maybe here?")
    ep = mne.Epochs(eeg, mne_events, tmin =  start, tmax = stop, baseline = None, preload = True, on_missing = 'ignore', verbose = False)
    ep._data = ep._data * 1000000
    ep.set_eeg_reference('average', projection = False)
    ep.filter(l_freq = 0.5, h_freq = None, method = 'iir', iir_params = None)
    ep.pick_types(eeg= True, exclude = [])
    #do  i need this part? maybe not
    #ep.resample(500.0)
    '''Create an xarray version of epoch data'''
    x = TimeSeries(ep._data, dims=('events','channels','time'),
                              coords={'events':events.to_records(),
                                      'channels':ep.info['ch_names'],
                                      'time':ep.times,
                                      'samplerate':ep.info['sfreq']})
    return ep, x, mne_events



In [7]:
def Channel_Powers(subject, session, channels_list, eeg, buffer_samp, wave_number ,freqs, path, error_log_path, log = True):

    channels = channels_list

    def create_folder(path):
        if not os.path.exists(path):
            os.makedirs(path, exist_ok=True)

    for name, i in zip(channels, range(len(channels))):
        channel_eeg = eeg[:, i, :]
#         print(channel_eeg)
#         wave_pows = MorletWaveletFilter(channel_eeg, freqs = freqs, width = wave_number, output = 'power').filter()
        wave_pows = MorletWaveletFilter(freqs = freqs, width = wave_number, output = 'power').filter(timeseries = channel_eeg)
        wave_pows = wave_pows.data.astype(np.float32)
        i_wave_pows = wave_pows[:, :, buffer_samp:-buffer_samp]
        if log == True:
            i_wave_pows = np.log10(i_wave_pows)
        else:
            pass 

        print(str(name)+" saved.")

        folder = path.format(subject , session)
        create_folder(folder)


        save_numpy(i_wave_pows,folder, "{}_{}_mt_logRet_enc_pow_{}".format(subject ,session, name))

In [8]:
def Grab_Encoding_Power(subj, expmt, sessions, freqs, path, error_log_path, start = .4, stop = .75, wave_number = 5):
    subj = subj 
    expmt = expmt 
    sessions = sessions 
    rel_start = start 
    rel_stop = stop
    wave_number = wave_number 
    freqs = freqs 
    
    #experiment specific params 
    list_number = 26 
    list_index = np.arange(1,26)
    
    
    buffer_ms = (wave_number * (1000/freqs[0])) / 2
    sr = 2048.
    buffer_samp = int(np.ceil(buffer_ms * (sr/1000.)))
    buffer_minimized = buffer_ms/1000
    
    
    path = path 
    error_log_path = error_log_path 
    
    error_log = []
    
    print("subject: "+str(subj))
    
    
    
    for sess in sessions:
        print("running session "+str(sess))
        try:
            
            reader = CMLReader(subj, expmt, sess)
            events = reader.load("task_events")
            word_evs = events.query("type == 'WORD'")
            
            print("filtering events")
            filtered, first, second = filter_repetitions(word_evs, list_index)
            
            print("loading and processing eeg")
            raw = get_raw_eeg(subj, sess)
            epoch, x, mne_events = Create_Epoch(raw, filtered, rel_start+(-1*buffer_minimized), rel_stop+buffer_minimized)
            eeg = ButterworthFilter(x, freq_range = [58., 62.], filt_type = 'stop', order = 4).filter()
            
            print("setting channels")
            channels = eeg.channels 
            channels = np.ndarray.tolist(np.asarray(channels))
            
            print("computing and logging channel power")
            Channel_Powers(subj, sess ,channels, eeg, buffer_samp, wave_number,freqs, path, error_log_path)
            
            
            
            
        except Exception as e:
            print("error: ")
            print(e)
            error_log.append(e)
            
            
    save_numpy(error_log, "/scratch/brandon.katerman/RepFR/"+str(subj)+"/", 'error_log.txt' )

In [22]:
filtered

,contact_1,contact_2,label,is_stim_only,type_1,type_2,avg.region,avg.x,avg.y,avg.z,avg.corrected.region,avg.corrected.x,avg.corrected.y,avg.corrected.z,hcp.region,hcp.x,hcp.y,hcp.z,ind.region,ind.x,ind.y,ind.z,ind.corrected.region,ind.corrected.x,ind.corrected.y,ind.corrected.z,mni.region,mni.x,mni.y,mni.z,stein.region,stein.x,stein.y,stein.z,vox.region,vox.x,vox.y,vox.z
83,106,107,LB6-LB7,False,D,D,None,-57.547523,-24.026829,-23.860397,None,-57.547523,-24.026829,-23.860397,None,NaN,NaN,NaN,inferiortemporal,-50.37473,-9.19357,-8.830949,None,-50.37473,-9.19357,-8.830949,Left MTG middle temporal gyrus,-57.96460,-15.323450,-17.767224,None,None,None,None,None,347.0,249.0,122.0
84,107,108,LB7-LB8,False,D,D,None,-61.165491,-22.028498,-22.232468,None,-61.165491,-22.028498,-22.232468,None,NaN,NaN,NaN,middletemporal,-55.39343,-8.96442,-7.644324,None,-55.39343,-8.96442,-7.644324,Left MTG middle temporal gyrus,-64.34425,-14.404650,-16.513466,None,None,None,None,None,356.0,249.0,123.0
85,108,109,LB8-LB9,False,D,D,None,-65.458429,-20.192064,-18.554608,None,-65.458429,-20.192064,-18.554608,None,NaN,NaN,NaN,middletemporal,-60.65673,-8.85612,-6.958573,None,-60.65673,-8.85612,-6.958573,Left STG superior temporal gyrus,-69.08490,-13.489150,-15.001081,None,None,None,None,None,366.0,249.0,124.0
92,116,117,LA6-LA7,False,D,D,None,-50.777525,-7.999199,-31.863737,None,-50.777525,-7.999199,-31.863737,None,NaN,NaN,NaN,inferiortemporal,-46.50273,3.82768,-14.753614,None,-46.50273,3.82768,-14.753614,Left MTG middle temporal gyrus,-52.81985,-2.124035,-27.461274,None,None,None,None,None,341.0,224.0,120.0
93,117,118,LA7-LA8,False,D,D,None,-53.398891,-10.613172,-29.188873,None,-53.398891,-10.613172,-29.188873,None,NaN,NaN,NaN,middletemporal,-51.43888,4.24743,-15.150680,None,-51.43888,4.24743,-15.150680,Left MTG middle temporal gyrus,-59.57510,-2.748765,-29.158049,None,None,None,None,None,350.0,223.0,120.0
